# Active learning to augment racism dataset

In [307]:
# import libraries 
import pandas as pd
# pd take screen width
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
%matplotlib inline


In [308]:
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations = arabic_punctuations + english_punctuations


def remove_urls (text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return text


def remove_emails(text):
    text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", "",  text, flags=re.MULTILINE)
    return text

# def remove_emoji(text):
#     return emoji.get_emoji_regexp().sub(u'', text)

def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def normalization(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def remove_stopwords(text):
    filtered_sentence = [w for w in text.split() if not w in arabic_stopwords]
    return ' '.join(filtered_sentence)

def cleaning_content(line):
    if (isinstance(line, float)):
        return None
    line.replace('\n', ' ')
    line = remove_emails(line)
    line = remove_urls(line)
    line = remove_emoji(line)
    nline = [w if '@' not in w else 'USERID' for w in line.split()]
    line = ' '.join(nline)
    line = line.replace('RT', '').replace('<LF>', '').replace('<br />','').replace('&quot;', '').replace('<url>', '').replace('USERID', '')


    # add spaces between punc,
    line = line.translate(str.maketrans({key: " {0} ".format(key) for key in punctuations}))

    # then remove punc,
    translator = str.maketrans('', '', punctuations)
    line = line.translate(translator)

    line = remove_stopwords(line)
    line=remove_diacritics(normalization(line))
    if line != '':
        return line

def hasDigits(s):
    return any( 48 <= ord(char) <= 57  or 1632 <= ord(char) <= 1641 for char in s)


In [309]:
# read the data
df = pd.read_csv('../DataSet/manuelly_annotated_off.csv')
pool = pd.read_csv('../DataSet/final_dataset.csv', encoding='utf-16')

print(df.shape)
df.head()

(2133, 4)


,id,text,Majority_Label,label
0,1,البطل قاتل وجاذف بحياته لتحيا انت واطي عيب الشوم,1.0,Verbal abuse
1,2,انو غريب يوجعك راسك القواتجيه عاده بيكونو بلا راس,1.0,Racism
2,3,طاءفي روح اسال اهل قانا وهني بخبروك,1.0,Racism
3,4,غرد الجحش وظن باءنه حصانا تتكلم العوني بموضوعيه فيرد كبهيم حقير جربان اصله,1.0,Verbal abuse
4,5,الفلسطينيين واللبنانين والشاميين لكان انتو لليوم تشربوا بول البعير تاكلون بصحن وتبزقون منو,1.0,Racism


In [310]:
print(pool.shape)
pool.head()

(31757, 2)


,Comment,Majority_Label
0,وزير الخارجيه اللبناني جبران باسيل قال سلسله تغريداته عقب اختتام القمه العربيه التنمويه الاقتصاديه والاجتماعيه لمسنا تج,0.0
1,سوريه بلد الحضارات تربطها بعليه او بحيوان,0.0
2,اخي الحاج اذا شعرت انك محرجا الانتقادات لتصريح جبران باسيل داعي لان تهاجم المنتقدين,0.0
3,فيك تعيش بلا تكب فتن ليل نهار وبكره قلهم الموضوع السيد,0.0
4,البطل قاتل وجاذف بحياته لتحيا انت واطي عيب الشوم,1.0


In [311]:
# drop nan values
df.dropna(inplace=True)
pool.dropna(inplace=True)
print( f" le df : {len(df)} , len pool {len(pool)} ")

 le df : 2125 , len pool 31755 


In [312]:
# substract the data from the pool
pool = pool[~pool['Comment'].isin(df['text'])]

print(len(pool))

28895


In [313]:
# see nan values
print(pool.isnull().sum())

# drop nan values
pool.dropna(inplace=True)

# clean the data
pool['Comment'] = pool['Comment'].apply(cleaning_content)

Comment           0
Majority_Label    0
dtype: int64


In [346]:
racism = pd.read_csv('../DataSet/racism_semi_assisted_MNB+LR.csv')

In [380]:
racism = pd.concat([racism, racism_manually_annotated])

In [381]:
racism

,text,Majority_Label,label,id
0,كلي هواءمريضه مين حضرتك توصفي خطاب السيد شو محلك الاعراب,1.0,not racism,NaN
1,اكبر شعب كيتفرج فافلام البورنو المغرب هههههههه,1.0,not racism,NaN
2,حسن نصرالله تاج راسك بلا اخلاق وحده واطيه وحقيره مثلك بدها تسكر تمها وتحط ايدها راسها تذك,1.0,not racism,NaN
3,انتي عم تحكي ام نص لسان روحي انطمي,1.0,not racism,NaN
4,كلي رقاصات الاعلام,1.0,not racism,NaN
...,...,...,...,...
1679,وانت مين حضرتك؟ مدير ميناء مدينة درنة؟ ولا القائم بالاعمال السفير الليبي في تركيا جاي تدلي بتصريحك عن طريق قسم التعليقات في الجزيرة ؟؟!! اكاد اجزم بأنك من جماعة الاسد وليس لك علاقة بليبيا !!,1.0,Racism,13637.0
1694,حين حكام السعودية و تركيا و ايران وسوريا و مصر يبعثون اولادهم للقتال يمكنكم ان تفعلو مثلهم! ان ترسلو اولادكم قبل ذالك فهو خطاء، علكم تعلمون...,1.0,Racism,13652.0
1701,قرينة جديدة على عمالة هؤلاء الملاحدة المرتدين للغرب ولصهاينة . ولا غرر فإسرائيل الكبرى المخطط لها تجاور هؤلاء ومن حق الجار على جاره مناصرته .,1.0,Racism,13659.0
1702,البعض يلومون السعوديه والخليج بعدم ايواء الاجئين السوريين بصراحه انهم يكلفون خزينه المملكه الكثير من المال لكن لم تسألو انفسكم اين سيقضي الملك والحاشيه العطله الصيفيه صج الا ما اختشو ماتو‏!‏,1.0,Racism,13660.0


In [382]:
# show duplicates
racism[racism.duplicated(subset=['text'], keep=False)]

,text,Majority_Label,label,id
2,حسن نصرالله تاج راسك بلا اخلاق وحده واطيه وحقيره مثلك بدها تسكر تمها وتحط ايدها راسها تذك,1.0,not racism,NaN
7,ستري تصويتهم الاعوج ولن سيقودون الاسد الصهيونيللمحاكم الجنايات الدوليه سيدينونه ولان حاميحدود الصهاينه والثوره ستقوده للمحاكمه ورغم مجلس الامن الاستعماري,1.0,not racism,NaN
10,اطرف قرات عثرروا وثيه سريه منطقه ناءيه انها قصه كنز القرصان او مصباح علاء الدين,1.0,not racism,NaN
11,السيسي وعباس كبار الصهاينه العرب وعملاء اسراءيل,1.0,Racism,NaN
13,لاحول لاقوه الا بالله الله تقبله الشهداء اهل مصر عليكم بالاخوان اخواتهم فهم اساس فساد تخلف الدين بريء منهم الي يوم الدين,1.0,Racism,NaN
...,...,...,...,...
379,ارحمك عرص خول تميم قنوات قطر الشاذه امثالها خيانه الشعوب العربيه الاسلاميه الافريقيه اعلاميون شواذهما هعدم الخونه بامري,1.0,Racism,2327.0
390,حرام كانوا يتنمروا عليها لانها طويله وهبيله وبتقولها بكل تاثر وانفعاليه,1.0,Racism,3564.0
392,ديما اكتر وحده بتتحمل مسءوليه باللي عم تتعرضلوا البنت انت وحده متنمره ومءذيه وحاجتك تكتبي وتمحي بتعرفي,1.0,Racism,3668.0
396,صارت ام نص لسان تتنمر تاج راسها شو محلها الاعراب هالتافهه لتتناول رءيس جمهوريه,1.0,Racism,3852.0


In [383]:
# drop duplicates
racism.drop_duplicates(subset=['text'], keep='first', inplace=True)


In [384]:
racism.label.value_counts()

Racism        289
not racism    221
Name: label, dtype: int64

In [385]:
# if label with no racism are less than Racism, we add some of no racism
if len(racism[racism['label'] == "not racism"]) < len(racism[racism['label'] == 'Racism']):
    # get the number of no racism to add
    nb = len(racism[racism['label'] == 'Racism']) - len(racism[racism['label'] == "not racism"])
    # get the no racism
    no_racism = df[df['label'] != 'Racism']
    #sample the no racism
    no_racism = no_racism.sample(nb)
    # add the label no racism
    no_racism['label'] = 'not racism'
    # concat the data
    racism = pd.concat([racism, no_racism], ignore_index=True)

In [353]:
if len(racism[racism['label'] == "not racism"]) > len(racism[racism['label'] == 'Racism']):
    # balance the data
    racism = racism.groupby('label').apply(lambda x: x.sample(len(racism[racism['label'] == 'Racism']), replace=True)).reset_index(drop=True)

In [320]:
# show duplicates
pool[pool.duplicated(subset=['Comment'], keep=False)]


,Comment,Majority_Label
0,وزير الخارجيه اللبناني جبران باسيل قال سلسله تغريداته عقب اختتام القمه العربيه التنمويه الاقتصاديه والاجتماعيه لمسنا تج,0.0
1,سوريه بلد الحضارات تربطها بعليه او بحيوان,0.0
2,اخي الحاج اذا شعرت انك محرجا الانتقادات لتصريح جبران باسيل داعي لان تهاجم المنتقدين,0.0
3,فيك تعيش بلا تكب فتن ليل نهار وبكره قلهم الموضوع السيد,0.0
5,ولعها وهيك بتكون الشقيقه قدمت اكبر خدمه لشقيقتها,0.0
...,...,...
31512,None,0.0
31526,الله اشافيك اختي,0.0
31557,None,0.0
31646,None,0.0


In [321]:
# drop duplicates
pool.drop_duplicates(subset=['Comment'], keep='first', inplace=True)


In [354]:
len(pool), len(racism)

(9553, 596)

In [355]:
# shuffle the data
racism = racism.sample(frac=1).reset_index(drop=True)

racism.head()

,text,Majority_Label,label,id
0,كلي هواءمريضه مين حضرتك توصفي خطاب السيد شو محلك الاعراب,1.0,not racism,NaN
1,اكبر شعب كيتفرج فافلام البورنو المغرب هههههههه,1.0,not racism,NaN
2,حسن نصرالله تاج راسك بلا اخلاق وحده واطيه وحقيره مثلك بدها تسكر تمها وتحط ايدها راسها تذك,1.0,not racism,NaN
3,انتي عم تحكي ام نص لسان روحي انطمي,1.0,not racism,NaN
4,كلي رقاصات الاعلام,1.0,not racism,NaN


In [324]:
# delete label racism from df
# df = df[df['label'] != 'Racism']

# concat racism and df
# df2 = pd.concat([df, racism])

# rename label diffrent than racism to 'not racism'
# df2['label'] = df2['label'].apply(lambda x: 'not racism' if x != 'Racism' else x)

# shuffle the data
# df2 = racism.sample(frac=1).reset_index(drop=True)

# balance the data
# df2 = df2.groupby('label').apply(lambda x: x.sample(len(racism), replace=True)).reset_index(drop=True)


In [386]:
# clean the data
df2 = racism
df2['text'] = df2['text'].apply(cleaning_content)

print(df2.label.value_counts())

df2

not racism    289
Racism        289
Name: label, dtype: int64


,text,Majority_Label,label,id
0,كلي هواءمريضه مين حضرتك توصفي خطاب السيد شو محلك الاعراب,1.0,not racism,NaN
1,اكبر شعب كيتفرج فافلام البورنو المغرب هههههههه,1.0,not racism,NaN
2,حسن نصرالله تاج راسك بلا اخلاق وحده واطيه وحقيره مثلك بدها تسكر تمها وتحط ايدها راسها تذك,1.0,not racism,NaN
3,انتي عم تحكي ام نص لسان روحي انطمي,1.0,not racism,NaN
4,كلي رقاصات الاعلام,1.0,not racism,NaN
...,...,...,...,...
573,استغفرك ربي واتوب اليك اعود باللله الشيطان الرجيم حشومه اش هادشي تانسمعو بقات حشمه ربي السلامه تكونو مغاربه شوهتو بينا الله يهديكم الطريق المستقيم بزاف هادشي الله يبعد علينا هاد الوباء ويبعد اعلينا هاد العفن تانشوف بلوك بلوك بلوك بلوك بلوك بلوك,1.0,not racism,9456.0
574,اساسا المافيات الميليشيات ادوات امريكا وايران للسيطره العراق ونهب موارده وقتل كوادره بيع وشراء الرتب والمناصب امر عادي عراق الدم قراطيه موجود سور للحراميه وليس للوطن,1.0,not racism,13151.0
575,يخرب بيتك ابشعك,1.0,not racism,698.0
576,لهجه شاويه فصحي,1.0,not racism,443.0


In [358]:
# delete text from pool that are in df2
pool = pool[~pool['Comment'].isin(df2['text'])]

# delete repeated text in pool
pool = pool.drop_duplicates(subset=['Comment'], keep='first')


In [359]:
len(pool)

9392

In [387]:
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2['text'], df2['label'], test_size=0.2, random_state=42)

# vectorize the data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# train the model
from sklearn.naive_bayes import MultinomialNB

MNB_model = MultinomialNB().fit(X_train, y_train)

# test the model
predicted_mnb = MNB_model.predict(X_test_tfidf)

# evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_test, predicted_mnb))


              precision    recall  f1-score   support

      Racism       0.55      0.69      0.61        54
  not racism       0.65      0.52      0.58        62

    accuracy                           0.59       116
   macro avg       0.60      0.60      0.59       116
weighted avg       0.61      0.59      0.59       116



In [388]:
# print probabilities
probs_mnb = MNB_model.predict_proba(X_test_tfidf)

In [390]:
# Trying svm
from sklearn import svm
svm_model = svm.SVC(kernel='linear', C=1, probability=True).fit(X_train, y_train)
predicted_svm = svm_model.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted_svm))

# print probabilities
probs_svm = svm_model.predict_proba(X_test_tfidf)


              precision    recall  f1-score   support

      Racism       0.56      0.63      0.59        54
  not racism       0.64      0.56      0.60        62

    accuracy                           0.59       116
   macro avg       0.60      0.60      0.59       116
weighted avg       0.60      0.59      0.60       116



In [389]:
# Trying logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
predicted_logreg = logreg.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted_logreg))

# print probabilities
probs_logreg = logreg.predict_proba(X_test_tfidf)

              precision    recall  f1-score   support

      Racism       0.59      0.65      0.62        54
  not racism       0.67      0.61      0.64        62

    accuracy                           0.63       116
   macro avg       0.63      0.63      0.63       116
weighted avg       0.63      0.63      0.63       116



In [391]:
# see the disagreement between the models on the test set and the actual labels
disagreement = pd.DataFrame({'text': X_test, 'actual': y_test, 'predicted_mnb': predicted_mnb, 'predicted_svm': predicted_svm, 'predicted_logreg': predicted_logreg})

In [392]:
disagreement

,text,actual,predicted_mnb,predicted_svm,predicted_logreg
257,مجرد راي فكرت نيوزيلاندا تسليم الارهابي الي تركيا لاعدامه صدق او تصدق بان دول استحماريه عربيه عميله تتدخل لانقاذ المجرم وهي مستعده لدفع الغالي والنفيس تسقط كرامتها باقي دول العالم ونحن نعرفهم معرفه الكلاب,Racism,Racism,not racism,not racism
574,اساسا المافيات الميليشيات ادوات امريكا وايران للسيطره العراق ونهب موارده وقتل كوادره بيع وشراء الرتب والمناصب امر عادي عراق الدم قراطيه موجود سور للحراميه وليس للوطن,not racism,Racism,Racism,Racism
485,اظن الشخصيه القذافيه عدوي انتقلت الي الجميع الصهاينه,Racism,Racism,Racism,Racism
101,صحيح نعال انتم يولي عليكم دحلان الزفت الي عزمي قذاره,not racism,Racism,Racism,Racism
566,مك اكبر غلط دارت فحياتها منين سمحات ليك دير المسرح كون زيرات معاك السمطه كون راك داير شي فاءده فحياتك,not racism,not racism,not racism,not racism
...,...,...,...,...,...
346,كالعاده الكويت لايطبق القانون الا حق اهل السنه لابسط زله واما الشيعه فيخالفوا وينتهكوا ويضربوا القوانين عرض الحاءط ولامن سامع او مطبق للقانون يبدو ان الشيعه اصحاب القرار الكويت,not racism,Racism,Racism,Racism
182,ههههههههههههه الله يلعن ابوك عهالخلفه العمي اسالك واجلطك وكمخ وزنخ وسميك ومقرف تجيك جلطه بنص لسان,not racism,not racism,not racism,not racism
442,هييه ارادوا اعتقاله لاعتقلوه فهو ارض وحكم الغرب جنوب افريقيا اكثرها سود,Racism,Racism,Racism,Racism
75,القبيحه اشترت قطر جاءزه نوبل والان تتسكع تركيا,not racism,not racism,not racism,not racism


In [393]:
# search for a text that is predicted as racism by all the models and is actually not racism

possible_faults = disagreement[(disagreement['predicted_mnb'] == 'Racism') & (disagreement['predicted_svm'] == 'Racism') & (disagreement['predicted_logreg'] == 'Racism') & (disagreement['actual'] == 'not racism')]

possible_faults


,text,actual,predicted_mnb,predicted_svm,predicted_logreg
574,اساسا المافيات الميليشيات ادوات امريكا وايران للسيطره العراق ونهب موارده وقتل كوادره بيع وشراء الرتب والمناصب امر عادي عراق الدم قراطيه موجود سور للحراميه وليس للوطن,not racism,Racism,Racism,Racism
101,صحيح نعال انتم يولي عليكم دحلان الزفت الي عزمي قذاره,not racism,Racism,Racism,Racism
408,الربيع العربيهوالتحرر منك روسياومن امريكاوايران اسراءيل والعملاءالحكام العرب وعدو واحدلشعوبناالعربيه وسيصل لروسيا وطهران امين,not racism,Racism,Racism,Racism
268,فعلا يتعلم العرب القنوات الرافضه للانقلاب تامل خيرا راعيه عاصفه الحزم,not racism,Racism,Racism,Racism
333,قناه المنار صدق تحدثت السيطره جرود عسال الورد تتحدثون سيطره النظام عليها وهي البلده وليست الجرود,not racism,Racism,Racism,Racism
272,لايمكن اسراءيل المعاهده تتجرا يمكن حوثيين مندسين يكرهون اسراءيل وهدفهم شق تحالف الحسم,not racism,Racism,Racism,Racism
76,اولا قرارك وليس قرار الشعب السوداني ولست شرعي ولتشترك بالحرب عملاء امريكا اليمنيين,not racism,Racism,Racism,Racism
10,اطرف قرات عثرروا وثيه سريه منطقه ناءيه انها قصه كنز القرصان او مصباح علاء الدين,not racism,Racism,Racism,Racism
81,ولك انت دب العالم سكوت,not racism,Racism,Racism,Racism
540,هاد العاءله جالس وسط منهوم خص يدلمهوم الحبس ليهوم العينين لفلوس او عارفينو مزوج او عاطينو البنت بلا عقد او ساهلاش هدي ناشي بنت صغيره هادي حرايفيه عندها ديبلوم او ديبلومات خصهوم الحبس بجوج او والديها نصابه ماشي مسلمين,not racism,Racism,Racism,Racism


In [394]:
pool.head()

pool = pool[pool['Majority_Label'] == 1]

In [395]:
pool.head()

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,prediction_logreg,probability_logreg
55,اودع مليار عشان حزب الشيطان بلبنان يمر بمرحله صعبه توقف الدعم الايران اثر الحصار عليها,1.0,Racism,0.594397,Racism,0.637042,Racism,0.897005
1102,خوش معارض واطي والله,1.0,not racism,0.543918,not racism,0.721535,not racism,0.960468
1150,تعلق الشعب الخري بطريقه الشعب الخري,1.0,Racism,0.630023,Racism,0.768728,Racism,0.980528
1166,جد جحاش القوات اذار نهيق نهيق,1.0,not racism,0.562330,not racism,0.795545,not racism,0.971576
1209,عنده كرامه طمر حاله بالتراب مجرم,1.0,Racism,0.537471,Racism,0.548064,Racism,0.561500


In [396]:
# predict the df
predicted_pool_MNB = MNB_model.predict(vectorizer.transform(pool['Comment']))
predicted_pool_prob_MNB = MNB_model.predict_proba(vectorizer.transform(pool['Comment']))

# add the prediction to the pool
pool['prediction_MNB'] = predicted_pool_MNB
# take the max probability
pool['probability_MNB'] = [max(x) for x in predicted_pool_prob_MNB]

# predict the df
predicted_pool_svm = svm_model.predict(vectorizer.transform(pool['Comment']))
predicted_pool_prob_svm = svm_model.predict_proba(vectorizer.transform(pool['Comment']))

# add the prediction to the pool
pool['prediction_svm'] = predicted_pool_svm
# take the max probability
pool['probability_svm'] = [max(x) for x in predicted_pool_prob_svm]

# predict the df
predicted_pool_logreg = logreg.predict(vectorizer.transform(pool['Comment']))
predicted_pool_prob_logreg = logreg.predict_proba(vectorizer.transform(pool['Comment']))

# add the prediction to the pool
pool['prediction_logreg'] = predicted_pool_logreg
# take the max probability
pool['probability_logreg'] = [max(x) for x in predicted_pool_prob_logreg]


pool.head()

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,prediction_logreg,probability_logreg
55,اودع مليار عشان حزب الشيطان بلبنان يمر بمرحله صعبه توقف الدعم الايران اثر الحصار عليها,1.0,Racism,0.539940,Racism,0.544033,Racism,0.799035
1102,خوش معارض واطي والله,1.0,not racism,0.604012,not racism,0.849992,not racism,0.999437
1150,تعلق الشعب الخري بطريقه الشعب الخري,1.0,Racism,0.631479,Racism,0.651690,Racism,0.968060
1166,جد جحاش القوات اذار نهيق نهيق,1.0,not racism,0.578248,not racism,0.715343,not racism,0.979499
1209,عنده كرامه طمر حاله بالتراب مجرم,1.0,Racism,0.516084,not racism,0.534595,not racism,0.695545


In [397]:

racist_comments = pool[pool['prediction_MNB'] == 'Racism']

racist_comments = racist_comments[racist_comments['probability_MNB'] > 0.65]
racist_comments = racist_comments[racist_comments['probability_svm'] > 0.85]

print(len(racist_comments))
# add column to racist comments called final_label
racist_comments['final_label'] = 'Nan'

racist_comments

61


,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,prediction_logreg,probability_logreg,final_label
1475,غريبين اهل الجزيره العربيه بتجوا لتعايروا بشكله بتشبهوه لكاءن ظريف الشكل والاخلاق ريتكم تتعلموا الكلاب وتصيروا ظريفين بدال الصلف والجلافه والاستعلاء,1.0,Racism,0.734295,Racism,0.856071,Racism,0.999909,Nan
3098,حشره سودانيه اخوانيه مهمتك نشر الكذب العهر حذاء ايراني تموله قطراءيل,1.0,Racism,0.731563,Racism,0.906466,Racism,0.999990,Nan
5671,علي العالم يءدب الدوله المجنونه ايران ويوقف قرصنتها لسفن العالم المياه الدوليه ايران تمارس الان مامارسه هتلر زمانه عندما ازعج الدنيا واضر بالسلم العالمي علي امريكا حليفه العالم والعدل الدولي توقف مجاملتها وانبطاحها لهذا النظام المجرم المهدد للسلم العالمي,1.0,Racism,0.707874,Racism,0.881912,Racism,0.999939,Nan
5687,وفيه خولات مصريين عايمين عوم السعوديه وبيقولك ايران السبب ايران اكتر دوله ماتلها ناس المصيبه دي التعريص السني الوسطي الجميل,1.0,Racism,0.724992,Racism,0.906928,Racism,0.999989,Nan
5690,لله وانا اليه راجعون اللهم ارحمهم ورزقهم الفردوس الاعلي الجنه اللهم احفظ مصر العروبه والناس الطيبه وكل رجالات مصر حكومه وشعب وساءر بلاد المسلمين والناس الطيبه ورزق ابناء مصر واهلها حب مصر والحرص عليها اللهم بكل خاءن قذر يريد الفتن المسلمين والناس الطيبه,1.0,Racism,0.681769,Racism,0.864770,Racism,0.999923,Nan
...,...,...,...,...,...,...,...,...,...
28625,ابت الحقاره ان تفارق اهلها ان الحقاره للحقير وسام فعلا انذل منك تري عيني قط واقذر منكم تلد النساء لحكام الخليج ولكو راضعين ثدي صهيونيه بتكونوا بالدناءه هاي اعوذ بالله الجاهل الساذج الخاين الي واقف ال سعود الامارات عقل فكروا شوي شغلوا هالطنجره خيانتكم قذارتكم بلشت احب ايران اقذر هيك حال وصلوه العرب عبيد امريكا,1.0,Racism,0.718402,Racism,0.884071,Racism,0.999893,Nan
28844,المجاهدين تركوا تحرير فلسطين ولاصحابك قل بربك ذهبت لقتال الصهاينه ابوك المعمم اللذي تتبعه اذا الجواب فلاتزايد المجاهدين,1.0,Racism,0.741814,Racism,0.871746,Racism,0.999969,Nan
29241,الب لاد كله مزرع ه شخصي ه لعن تره ماءه هواءه نس اءها حقوله المخضوضره البناي ات يس نتره الش بابيك علي وره لع نتره المي ادين تحم اس نتره نتره يق يم ثي ابن ربط الخ,1.0,Racism,0.703662,Racism,0.855883,Racism,0.999903,Nan
29343,وصنعتها االموسادوكلما شعروا بسقوط حكام اليهود مصر والامه,1.0,Racism,0.706583,Racism,0.864711,Racism,0.999949,Nan


In [398]:
from IPython.display import clear_output
# manually check if the comments are racist or not by prompting the user to enter 1 or 0
for i in range(len(racist_comments)):

    print(f'comment {i} / text = {racist_comments.iloc[i]["Comment"]} , MNB proba = {racist_comments.iloc[i]["probability_MNB"]} , svm proba = {racist_comments.iloc[i]["probability_svm"]}')
    print('---------------------------------------')

    inp = input(f'{racist_comments.iloc[i]["Comment"]} : ')
    if inp == '1':

        racist_comments.iloc[i, racist_comments.columns.get_loc('final_label')] = 'Racism'

        clear_output()
    elif inp == '0':
        racist_comments.iloc[i, racist_comments.columns.get_loc('final_label')] = 'not racism'
        clear_output()
    elif inp == 'exit':
        break

    print('---------------------------------------')

---------------------------------------
comment 17 / text = اين طرف اخر الخنزيره تري ان المشاهد ساذيج يعرف يجري اليمن لقطاء البشريه مضيف اربعه كلاب ينبح بسنفونيه ال سلولي قطراءيليه تفووووووو شرفكم , MNB proba = 0.7063676745310826 , svm proba = 0.8549990699591293
---------------------------------------


In [399]:
racist_comments

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,prediction_logreg,probability_logreg,final_label
1475,غريبين اهل الجزيره العربيه بتجوا لتعايروا بشكله بتشبهوه لكاءن ظريف الشكل والاخلاق ريتكم تتعلموا الكلاب وتصيروا ظريفين بدال الصلف والجلافه والاستعلاء,1.0,Racism,0.734295,Racism,0.856071,Racism,0.999909,Racism
3098,حشره سودانيه اخوانيه مهمتك نشر الكذب العهر حذاء ايراني تموله قطراءيل,1.0,Racism,0.731563,Racism,0.906466,Racism,0.999990,Racism
5671,علي العالم يءدب الدوله المجنونه ايران ويوقف قرصنتها لسفن العالم المياه الدوليه ايران تمارس الان مامارسه هتلر زمانه عندما ازعج الدنيا واضر بالسلم العالمي علي امريكا حليفه العالم والعدل الدولي توقف مجاملتها وانبطاحها لهذا النظام المجرم المهدد للسلم العالمي,1.0,Racism,0.707874,Racism,0.881912,Racism,0.999939,not racism
5687,وفيه خولات مصريين عايمين عوم السعوديه وبيقولك ايران السبب ايران اكتر دوله ماتلها ناس المصيبه دي التعريص السني الوسطي الجميل,1.0,Racism,0.724992,Racism,0.906928,Racism,0.999989,not racism
5690,لله وانا اليه راجعون اللهم ارحمهم ورزقهم الفردوس الاعلي الجنه اللهم احفظ مصر العروبه والناس الطيبه وكل رجالات مصر حكومه وشعب وساءر بلاد المسلمين والناس الطيبه ورزق ابناء مصر واهلها حب مصر والحرص عليها اللهم بكل خاءن قذر يريد الفتن المسلمين والناس الطيبه,1.0,Racism,0.681769,Racism,0.864770,Racism,0.999923,not racism
...,...,...,...,...,...,...,...,...,...
28625,ابت الحقاره ان تفارق اهلها ان الحقاره للحقير وسام فعلا انذل منك تري عيني قط واقذر منكم تلد النساء لحكام الخليج ولكو راضعين ثدي صهيونيه بتكونوا بالدناءه هاي اعوذ بالله الجاهل الساذج الخاين الي واقف ال سعود الامارات عقل فكروا شوي شغلوا هالطنجره خيانتكم قذارتكم بلشت احب ايران اقذر هيك حال وصلوه العرب عبيد امريكا,1.0,Racism,0.718402,Racism,0.884071,Racism,0.999893,Nan
28844,المجاهدين تركوا تحرير فلسطين ولاصحابك قل بربك ذهبت لقتال الصهاينه ابوك المعمم اللذي تتبعه اذا الجواب فلاتزايد المجاهدين,1.0,Racism,0.741814,Racism,0.871746,Racism,0.999969,Nan
29241,الب لاد كله مزرع ه شخصي ه لعن تره ماءه هواءه نس اءها حقوله المخضوضره البناي ات يس نتره الش بابيك علي وره لع نتره المي ادين تحم اس نتره نتره يق يم ثي ابن ربط الخ,1.0,Racism,0.703662,Racism,0.855883,Racism,0.999903,Nan
29343,وصنعتها االموسادوكلما شعروا بسقوط حكام اليهود مصر والامه,1.0,Racism,0.706583,Racism,0.864711,Racism,0.999949,Nan


In [400]:
# delete rows with column final_label = Nan
racist_comments = racist_comments[racist_comments['final_label'] != 'Nan']

racist_comments

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,prediction_logreg,probability_logreg,final_label
1475,غريبين اهل الجزيره العربيه بتجوا لتعايروا بشكله بتشبهوه لكاءن ظريف الشكل والاخلاق ريتكم تتعلموا الكلاب وتصيروا ظريفين بدال الصلف والجلافه والاستعلاء,1.0,Racism,0.734295,Racism,0.856071,Racism,0.999909,Racism
3098,حشره سودانيه اخوانيه مهمتك نشر الكذب العهر حذاء ايراني تموله قطراءيل,1.0,Racism,0.731563,Racism,0.906466,Racism,0.999990,Racism
5671,علي العالم يءدب الدوله المجنونه ايران ويوقف قرصنتها لسفن العالم المياه الدوليه ايران تمارس الان مامارسه هتلر زمانه عندما ازعج الدنيا واضر بالسلم العالمي علي امريكا حليفه العالم والعدل الدولي توقف مجاملتها وانبطاحها لهذا النظام المجرم المهدد للسلم العالمي,1.0,Racism,0.707874,Racism,0.881912,Racism,0.999939,not racism
5687,وفيه خولات مصريين عايمين عوم السعوديه وبيقولك ايران السبب ايران اكتر دوله ماتلها ناس المصيبه دي التعريص السني الوسطي الجميل,1.0,Racism,0.724992,Racism,0.906928,Racism,0.999989,not racism
5690,لله وانا اليه راجعون اللهم ارحمهم ورزقهم الفردوس الاعلي الجنه اللهم احفظ مصر العروبه والناس الطيبه وكل رجالات مصر حكومه وشعب وساءر بلاد المسلمين والناس الطيبه ورزق ابناء مصر واهلها حب مصر والحرص عليها اللهم بكل خاءن قذر يريد الفتن المسلمين والناس الطيبه,1.0,Racism,0.681769,Racism,0.864770,Racism,0.999923,not racism
5886,اقتلو القطط والكلاب الضاله تنجو مصر,1.0,Racism,0.679273,Racism,0.856008,Racism,0.999981,not racism
17780,تعليقي مختصر لكل امه زباله وهءلاء زباله السنه ان اغلب نزحوا كانوا كلاب للحكومه الصفويه الله ناصر جنده وخاذل المرتدين,1.0,Racism,0.726834,Racism,0.875269,Racism,0.999911,Racism
17823,ادعي احرار مصر وشرفاءها الالتفات حول الدوله لانقاذ مصر وليس خيار اخر ان الانقلاب يستهدف الاحرارجميعا فهل يعي الاخوان الحقيقه فوات الاوان اني ناصح امين,1.0,Racism,0.798069,Racism,0.932812,Racism,1.000000,not racism
17987,هاي شكو الهوسه ايران مصر وشكو يعني الخره اخو البول كله نكاسه,1.0,Racism,0.742189,Racism,0.918236,Racism,0.999998,not racism
19161,اذا عربت خربت اينما حل الاعراب حل الحراب,1.0,Racism,0.788012,Racism,0.873296,Racism,0.999939,Racism


In [374]:
racist_comments.final_label.value_counts()

not racism    68
Racism        28
Name: final_label, dtype: int64

In [375]:
# delete the columns that we don't need
racist_comments.drop(['prediction_MNB', 'probability_MNB', 'prediction_svm', 'probability_svm', 'probability_logreg', 'prediction_logreg'], axis=1, inplace=True)
# rename the columns
racist_comments.rename(columns={'Comment': 'text', 'final_label': 'label'}, inplace=True)

racist_comments

C:\Users\makch\AppData\Local\Temp\ipykernel_6712\1634730582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racist_comments.drop(['prediction_MNB', 'probability_MNB', 'prediction_svm', 'probability_svm', 'probability_logreg', 'prediction_logreg'], axis=1, inplace=True)
C:\Users\makch\AppData\Local\Temp\ipykernel_6712\1634730582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racist_comments.rename(columns={'Comment': 'text', 'final_label': 'label'}, inplace=True)


,text,Majority_Label,label
1475,غريبين اهل الجزيره العربيه بتجوا لتعايروا بشكله بتشبهوه لكاءن ظريف الشكل والاخلاق ريتكم تتعلموا الكلاب وتصيروا ظريفين بدال الصلف والجلافه والاستعلاء,1.0,Racism
2208,للاسف طز بهذا البلد وطز بهيك نواب فعلا نحنا اللبنانيين شعب بيهوي,1.0,Racism
2268,علوي وصخ,1.0,not racism
2466,هيدا ايراني فارسي يهودي احفاد القرده الخنازير,1.0,Racism
2760,منافق وفار مز العداله,1.0,Racism
...,...,...,...
19846,ان الله غفور رحيم كلنا عندنا عيوبنا داك الشماته اللي صور الفيديو سير الله يفضحك دنيا واخيره,1.0,not racism
19969,الجراءم مدعومه حكام الخلبج,1.0,not racism
20612,اساسا انتم تدرون ان سلاطين اراضيكم الجاهليه ماضغطونا اظن والله اعلم ان كلمه ساضغط مشتقه ضغط الجزيره مرتفع وظهرت بسببه الكلمه,1.0,Racism
21750,مخلفات الدوله العبيديه مصر تغني الصيام والعباده جزء الشعاءر,1.0,Racism


In [376]:

f = pd.concat([df2, racist_comments])

f.to_csv('../DataSet//racism_semi_assisted_MNB+LR+SVM.csv', index=False)

f

,text,Majority_Label,label,id
0,كلي هواءمريضه مين حضرتك توصفي خطاب السيد شو محلك الاعراب,1.0,not racism,NaN
1,اكبر شعب كيتفرج فافلام البورنو المغرب هههههههه,1.0,not racism,NaN
2,حسن نصرالله تاج راسك بلا اخلاق وحده واطيه وحقيره مثلك بدها تسكر تمها وتحط ايدها راسها تذك,1.0,not racism,NaN
3,انتي عم تحكي ام نص لسان روحي انطمي,1.0,not racism,NaN
4,كلي رقاصات الاعلام,1.0,not racism,NaN
...,...,...,...,...
19846,ان الله غفور رحيم كلنا عندنا عيوبنا داك الشماته اللي صور الفيديو سير الله يفضحك دنيا واخيره,1.0,not racism,NaN
19969,الجراءم مدعومه حكام الخلبج,1.0,not racism,NaN
20612,اساسا انتم تدرون ان سلاطين اراضيكم الجاهليه ماضغطونا اظن والله اعلم ان كلمه ساضغط مشتقه ضغط الجزيره مرتفع وظهرت بسببه الكلمه,1.0,Racism,NaN
21750,مخلفات الدوله العبيديه مصر تغني الصيام والعباده جزء الشعاءر,1.0,Racism,NaN
